## EmpathyBot – Notebook Overview

This notebook demonstrates building a personalized empathetic chatbot using transformer-based emotion models and semantic embeddings. The chatbot predicts the emotion of user input, retrieves similar texts from a dataset, highlights key words, and generates an empathetic response.

## 📦 Install Required Packages

In [1]:
!pip install -q transformers sentence-transformers faiss-cpu streamlit fastapi uvicorn pandas nltk pyngrok
!pip install gtts soundfile librosa
!apt-get update && apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://arc

## Load the Emotion Dataset

In [2]:
from datasets import load_dataset
import pandas as pd

# Load emotion dataset
ds = load_dataset("cardiffnlp/tweet_eval", "emotion")

# Convert to DataFrame
df_train = pd.DataFrame(ds["train"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

In [3]:
df_train

,text,label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3
...,...,...
3252,I get discouraged because I try for 5 fucking ...,3
3253,The @user are in contention and hosting @user ...,3
3254,@user @user @user @user @user as a fellow UP g...,0
3255,You have a #problem? Yes! Can you do #somethin...,0


## 🎯 Filter Important Emotion Labels

The dataset contains multiple emotion labels, but for our analysis and chatbot, we focus only on Positive (1) and Negative (3) emotions. Neutral and other emotions are ignored for simplicity.

| Label | Meaning (example)                |
| ----- | -------------------------------- |
| 0     | Neutral / No strong emotion      |
| 1     | Positive / Happy / Cute          |
| 2     | Caring / Empathetic / Supportive |
| 3     | Negative / Sad / Worried / Angry |


In [4]:
# Keep only Positive (1) and Negative (3) labels
df = df_train[df_train['label'].isin([1, 3])]
df.head()

,text,label
2,No but that's so cute. Atsu was probably shy a...,1
4,it's pretty depressing when u hit pan on ur fa...,3
6,Making that yearly transition from excited and...,3
7,Tiller and breezy should do a collab album. Ra...,1
11,#NewYork: Several #Baloch &amp; Indian activis...,3


## Text Preprocessing Step

Before feeding text data into the model, it's important to clean and normalize it. Preprocessing helps the model focus on meaningful content and reduces noise from URLs, mentions, or extra spaces.

In [5]:
import re

def preprocess_text(text):
    """
    Preprocess text without removing emojis.
    """
    text = text.lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_text'] = df['text'].apply(preprocess_text)
df.drop(columns=["text"], inplace=True)
df.head()

/tmp/ipython-input-879956980.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['text'].apply(preprocess_text)
/tmp/ipython-input-879956980.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["text"], inplace=True)


,label,clean_text
2,1,no but that's so cute. atsu was probably shy a...
4,3,it's pretty depressing when u hit pan on ur fa...
6,3,making that yearly transition from excited and...
7,1,tiller and breezy should do a collab album. ra...
11,3,#newyork: several #baloch &amp; indian activis...


## Load Pre-trained Emotion Model

To predict emotions (happiness/sadness) from text, I use a pre-trained BERT-based sentiment model.
--
 `The nlptown/bert-base-multilingual-uncased-sentiment` model is capable of predicting sentiment across multiple languages and returns scores for 1–5 star ratings. We map these ratings to binary emotion labels (Positive → Happiness, Negative → Sadness).

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

def create_emotion_model(model_name="nlptown/bert-base-multilingual-uncased-sentiment"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    classifier = pipeline(
        "text-classification",
        model=model,
        tokenizer=tokenizer,
        return_all_scores=True
    )
    return classifier

emotion_model = create_emotion_model()

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## Test the Emotion Model

In [7]:
# Test
text = "I feel really happy today"
results = emotion_model(text)
for r in results[0]:
    print(f"Emotion: {r['label']}, Score: {r['score']:.4f}")

Emotion: 1 star, Score: 0.0033
Emotion: 2 stars, Score: 0.0035
Emotion: 3 stars, Score: 0.0247
Emotion: 4 stars, Score: 0.2547
Emotion: 5 stars, Score: 0.7138


## Binary Emotion Mapping

1–2 stars → Sadness (negative)

3–5 stars → Happiness (positive)

In [8]:
# 1–2 stars → negative (3), 3–5 stars → positive (1)
def predict_binary_label_star(text, model):
    results = model(text)[0]
    top_star = max(results, key=lambda x: x['score'])['label']
    star_num = int(top_star.split()[0])

    return 3 if star_num <= 2 else 1

In [9]:
# Apply to dataset
df['predicted_label'] = df['clean_text'].apply(lambda x: predict_binary_label_star(x, emotion_model))
df['predicted_emotion'] = df['predicted_label'].map({1: "happiness", 3: "sadness"})
df.head()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/tmp/ipython-input-2759358882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_label'] = df['clean_text'].apply(lambda x: predict_binary_label_star(x, emotion_model))
/tmp/ipython-input-2759358882.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_emotion'] = df['predicted_label'].map({1: "happiness", 3: "sadness"})


,label,clean_text,predicted_label,predicted_emotion
2,1,no but that's so cute. atsu was probably shy a...,1,happiness
4,3,it's pretty depressing when u hit pan on ur fa...,1,happiness
6,3,making that yearly transition from excited and...,1,happiness
7,1,tiller and breezy should do a collab album. ra...,3,sadness
11,3,#newyork: several #baloch &amp; indian activis...,3,sadness


## Dealing With Speech Data

### 1. Install dependencies

In [30]:
# 📦 Install required packages
!pip install sounddevice wavio librosa transformers sentence-transformers gTTS --quiet
!apt-get update && apt-get install -y portaudio19-dev --quiet

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

### 🎙️ 2. Record Audio

In [52]:
from google.colab import output
from base64 import b64decode
from IPython.display import display, HTML
import time

recorded_audio = {}

def _record_callback(data):
    recorded_audio["data"] = data

output.register_callback("notebook.record", _record_callback)

def record_audio_colab(duration=2, filename="input.wav"):
    recorded_audio.clear()

    display(HTML(f"""
      <div id="recorder-ui">🎤 Recording for {duration} seconds...</div>
      <script>
        async function record() {{
          const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
          const recorder = new MediaRecorder(stream);
          let data = [];
          recorder.ondataavailable = event => data.push(event.data);
          recorder.onstop = () => {{
            let audio = new Blob(data, {{ type: 'audio/wav' }});
            let reader = new FileReader();
            reader.readAsDataURL(audio);
            reader.onloadend = () => {{
              google.colab.kernel.invokeFunction('notebook.record', [reader.result], {{}});
            }};
          }};
          recorder.start();
          setTimeout(() => {{
            recorder.stop();
            stream.getTracks().forEach(track => track.stop());
            document.getElementById("recorder-ui").innerHTML = "Recording finished. Run the next cell.";
          }}, {duration * 1000});
        }}
        record();
      </script>
    """))

    # Wait until JS callback sends audio
    while "data" not in recorded_audio:
        time.sleep(0.1)

    # Save audio
    audio = b64decode(recorded_audio["data"].split(",")[1])
    with open(filename, "wb") as f:
        f.write(audio)
    return filename

# ▶️ Record 10 seconds
audio_path = record_audio_colab(duration=2)
print(" Saved audio:", audio_path)

In [ ]:
from IPython.display import Audio
Audio(audio_path)

### 📝 3. Transcribe with Wav2Vec2

In [ ]:
import torch, librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
stt_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

def transcribe_audio(path):
    audio, sr = librosa.load(path, sr=16000)
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        logits = stt_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    return processor.batch_decode(predicted_ids)[0]

# 📝 Transcribe
text = transcribe_audio(audio_path)
print("🗣️ Transcribed Text:", text)


### 😀 4. Predict Emotion

In [ ]:
# Using your existing function predict_binary_label_star
emotion = predict_binary_label_star(text, emotion_model)
emotion = "happiness" if emotion == 1 else "sadness"
print("😀 Predicted Emotion:", emotion)


### 🔊 5. Text-to-Speech (TTS)

In [ ]:
from gtts import gTTS
from IPython.display import Audio, display

def speak_text(text, filename="reply.mp3"):
    tts = gTTS(text=text, lang="en")
    tts.save(filename)
    return filename

# 💡 Empathetic Reply
reply = f"I hear you. It sounds like you're feeling {emotion}. You're not alone 💛"
print("💡 Empathetic Reply:", reply)

# 🔊 Speak Reply
tts_file = speak_text(reply)
display(Audio(tts_file, autoplay=True))

## Evaluating Model Performance

This cell evaluates how well your binary emotion mapping (Happiness vs Sadness) aligns with the original dataset labels.

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(df['label'], df['predicted_label'], labels=[1,3])
print("Confusion Matrix:\n", cm)

report = classification_report(df['label'], df['predicted_label'], labels=[1,3])
print("Classification Report:\n", report)

matches = (df['label'] == df['predicted_label']).sum()
total = len(df)
print(f"Exact matches: {matches}/{total} ({matches/total:.2%})")


Confusion Matrix:
 [[514 194]
 [237 618]]
Classification Report:
               precision    recall  f1-score   support

           1       0.68      0.73      0.70       708
           3       0.76      0.72      0.74       855

    accuracy                           0.72      1563
   macro avg       0.72      0.72      0.72      1563
weighted avg       0.73      0.72      0.72      1563

Exact matches: 1132/1563 (72.42%)


## Creating Embeddings and Building FAISS Index

In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode texts
texts = df['clean_text'].tolist()
text_embeddings = embed_model.encode(texts, convert_to_tensor=True)

# FAISS index
dimension = text_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(text_embeddings.cpu()))

# Map label to emotion
label_to_emotion = {1: "happiness", 3: "sadness"}
df['predicted_emotion'] = df['predicted_label'].map(label_to_emotion)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2366333654.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_emotion'] = df['predicted_label'].map(label_to_emotion)


In [12]:
df["clean_text"].iloc[0], df["predicted_emotion"].iloc[0]

("no but that's so cute. atsu was probably shy about photos before but cherry helped her out uwu",
 'happiness')

## Retrieving Similar Responses and Generating Empathetic Replies

In [13]:
def retrieve_similar_responses(user_text, user_emotion, top_k=3):
    user_emb = embed_model.encode([user_text], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(user_emb, top_k*5)

    retrieved = []
    for idx in indices[0]:
        if df.iloc[idx]['predicted_emotion'] == user_emotion:
            retrieved.append(df.iloc[idx]['clean_text'])
        if len(retrieved) == top_k:
            break
    return retrieved

def generate_empathetic_reply(user_text, user_emotion, top_k=3):
    retrieved = retrieve_similar_responses(user_text, user_emotion, top_k=top_k)
    combined_reply = " ".join(retrieved)
    final_reply = f"{combined_reply} I’m not a therapist; please seek professional help for serious issues."
    return final_reply

# Test sample
sample = "heard of panic! at the disco? how about kach-ing! at the atm"
reply = generate_empathetic_reply(sample, "sadness")
print("Sample reply:", reply)

Sample reply: heard of panic! at the disco? how about kach-ing! at the atm i really want to go for fright night but i really don't 😁 anyyyyone wanna go to fright fest with me on friday night? 👻 I’m not a therapist; please seek professional help for serious issues.


## Testing the EmpathyBot on Sample Dataset Entries

In [14]:
# Sample test
for i, row in df.sample(5).iterrows():
    user_text = row['clean_text']
    predicted_emotion = row['predicted_emotion']

    reply = generate_empathetic_reply(user_text, predicted_emotion)

    print("User:", user_text)
    print("Predicted Emotion:", predicted_emotion)
    print("Bot Reply:", reply)
    print("Original Emotion:", label_to_emotion[row['label']])
    print("-"*50)


User: a3: but chronic sadness may mean there are underlying issues than getting sad occassionally over a particular issue (2/2) #mhchat
Predicted Emotion: sadness
Bot Reply: a3: but chronic sadness may mean there are underlying issues than getting sad occassionally over a particular issue (2/2) #mhchat indeed &amp; is sadness unavoidable? #mhchat interesting topic this evening... sadness &amp; low mood. #mhchat I’m not a therapist; please seek professional help for serious issues.
Original Emotion: sadness
--------------------------------------------------
User: it feels like there are no houses out there for us. with the most basic requirements i have, there are literally no options. #discouraged
Predicted Emotion: sadness
Bot Reply: it feels like there are no houses out there for us. with the most basic requirements i have, there are literally no options. #discouraged it feels like there are no houses out there for us. with the most basic requirements i have, there are literally no o

# 💛 EmpathyBot Streamlit App

This Streamlit app allows users to input a text prompt and receive a detailed **emotion analysis** along with an **empathetic response** based on a curated dataset. It is designed for both usability and visual appeal.

---

## **Features:**

- **Emotion prediction:** Detects whether the input text expresses *happiness* or *sadness* using a BERT-based sentiment model.
- **Model confidence visualization:** Displays a bar chart showing the confidence scores for all star ratings.
- **Similar texts from dataset:** Retrieves up to 3 examples from the dataset that match the predicted emotion.
- **Highlighted keywords:** Highlights words in the input text that strongly contribute to the predicted emotion.
- **Empathetic combined reply:** Provides a synthesized response based on the retrieved similar texts.

---

## **App Workflow:**

### 1. User Input
- The user types or pastes a text in the **centered text area**.

### 2. Emotion Prediction
- The text is fed into a pretrained `nlptown/bert-base-multilingual-uncased-sentiment` model.
- Predictions are mapped into **binary emotions**:
  - `1–2 stars → sadness`
  - `3–5 stars → happiness`

### 3. Keyword Highlighting
- Key emotion words (like *happy*, *sad*, *love*, *pain*) are highlighted with **color-coded backgrounds**.

### 4. Vector Retrieval
- The app encodes the input text using **SentenceTransformer embeddings**.
- Uses **FAISS** to find similar texts from the dataset that share the same emotion.

### 5. Empathetic Reply Generation
- Combines the similar texts into a **single empathetic response**.
- Displayed as a **bullet-point item** below the input.

### 6. Output Display
Shows:
- Highlighted input text
- Predicted emotion
- Similar dataset texts
- Empathetic combined reply
- Confidence score chart

---

## **UI/UX Design**
- Everything is **centered** using custom CSS.
- Input box and buttons are visually aligned.
- Highlighted words use **green** for happy and **red** for sad emotions.
- Bullet points summarize key outputs.


In [22]:
!pip install gtts librosa streamlit-mic-recorder
!apt-get update && apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 17.0 MB/s eta 0:00:00
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/so

In [53]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import re
import torch, librosa, tempfile
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from gtts import gTTS

# 🎤 Mic recorder
from streamlit_mic_recorder import mic_recorder

# --------------------------
# App Config
# --------------------------
st.set_page_config(
    page_title="EmpathyBot 💛",
    page_icon="💛",
    layout="wide"
)

# --------------------------
# Custom CSS for centering everything
# --------------------------
st.markdown("""
<style>
    .css-18e3th9 {
        display: flex;
        flex-direction: column;
        align-items: center;
    }
    .stTextInput>div>div>input, .stTextArea>div>div>textarea {
        text-align: center;
    }
    .stButton>button {
        margin-left: auto;
        margin-right: auto;
        display: block;
    }
    h1, h2, h3, h4, h5, h6, p, span, div, li {
        text-align: center !important;
    }
</style>
""", unsafe_allow_html=True)

st.title("💛 EmpathyBot")
st.markdown("""
**Enter your text and get:**
- Emotion prediction (Happiness / Sadness)
- Model confidence visualization
- Similar texts from dataset
- Highlighted key words contributing to emotion
- Empathetic combined reply
- Speech-to-Text + Text-to-Speech
""")

# --------------------------
# Load Dataset
# --------------------------
@st.cache_data
def load_dataset():
    from datasets import load_dataset
    ds = load_dataset("cardiffnlp/tweet_eval", "emotion")
    df = pd.DataFrame(ds["train"])
    df = df[df['label'].isin([1,3])]
    df['clean_text'] = df['text'].str.lower()
    df['clean_text'] = df['clean_text'].str.replace(r'http\S+|www.\S+', '', regex=True)
    df['clean_text'] = df['clean_text'].str.replace(r'@\w+', '', regex=True)
    df['clean_text'] = df['clean_text'].str.replace(r'\s+', ' ', regex=True).str.strip()
    df['predicted_emotion'] = df['label'].map({1:"happiness", 3:"sadness"})
    return df

df = load_dataset()

# --------------------------
# Load Models
# --------------------------
@st.cache_resource
def load_models():
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)
    embed_model = SentenceTransformer('all-MiniLM-L6-v2')
    return classifier, embed_model

emotion_model, embed_model = load_models()

# --------------------------
# Build FAISS Index
# --------------------------
@st.cache_resource
def build_faiss(_embed_model, df):
    embeddings = _embed_model.encode(df['clean_text'].tolist(), convert_to_tensor=True)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings.cpu()))
    return index

index = build_faiss(embed_model, df)

# --------------------------
# Helper Functions
# --------------------------
def predict_emotion(text, model):
    results = model(text)[0]
    scores = {r['label']: r['score'] for r in results}
    top_star = max(results, key=lambda x: x['score'])['label']
    star_num = int(top_star.split()[0])
    predicted_emotion = "happiness" if star_num >= 3 else "sadness"
    return predicted_emotion, scores

def highlight_text(text, emotion):
    happy_words = ["happy","love","good","great","fun","joy","excited"]
    sad_words = ["sad","bad","angry","upset","depressed","pain","worried"]
    words = text.split()
    highlighted = []
    for w in words:
        clean_w = re.sub(r'[^\w\s]', '', w.lower())
        if emotion=="happiness" and clean_w in happy_words:
            highlighted.append(f"<span style='background-color:#d4edda'>{w}</span>")
        elif emotion=="sadness" and clean_w in sad_words:
            highlighted.append(f"<span style='background-color:#f8d7da'>{w}</span>")
        else:
            highlighted.append(w)
    return " ".join(highlighted)

def retrieve_similar_texts(user_text, user_emotion, top_k=3):
    user_emb = embed_model.encode([user_text], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(user_emb, top_k*5)
    retrieved = []
    for idx in indices[0]:
        if df.iloc[idx]['predicted_emotion'] == user_emotion:
            retrieved.append(df.iloc[idx]['clean_text'])
        if len(retrieved) == top_k:
            break
    return retrieved

def generate_empathetic_reply(similar_texts):
    if similar_texts:
        combined = " ".join(similar_texts)
        return f"💡 Empathetic reply based on dataset:\n\n{combined}"
    else:
        return "💡 Sorry, no similar texts found in the dataset."

# --------------------------
# Speech-to-Text (STT)
# --------------------------
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
stt_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

def transcribe_audio(path):
    audio, sr = librosa.load(path, sr=16000)
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        logits = stt_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    return processor.batch_decode(predicted_ids)[0]

# --------------------------
# Text-to-Speech (TTS)
# --------------------------
def speak_text(text, filename="reply.mp3"):
    tts = gTTS(text=text, lang="en")
    tts.save(filename)
    return filename

# --------------------------
# User Input
# --------------------------
user_input = st.text_area("Enter your text:")

# 🎤 Upload audio
audio_file = st.file_uploader("🎤 Or upload your voice (wav/mp3)", type=["wav","mp3"])
if audio_file:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        tmp.write(audio_file.read())
        user_input = transcribe_audio(tmp.name)
        st.write("🗣️ Transcribed Text:", user_input)

# 🎤 Record audio
recorded_audio = mic_recorder(
    start_prompt="🎙️ Start Recording",
    stop_prompt="⏹️ Stop Recording",
    key="recorder"
)

if recorded_audio is not None:
    audio_bytes = recorded_audio["bytes"]  # ✅ Extract bytes
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        tmp.write(audio_bytes)
        tmp_path = tmp.name
    user_input = transcribe_audio(tmp_path)
    st.write("🗣️ Transcribed Text:", user_input)


# --------------------------
# Analysis
# --------------------------
if st.button("Analyze") and user_input.strip():
    predicted_emotion, scores = predict_emotion(user_input, emotion_model)

    # Highlight words
    highlighted_text = highlight_text(user_input, predicted_emotion)
    st.markdown(f"""
    <div style='text-align:center; padding:20px; border-radius:10px; background-color:#f0f8ff; font-size:18px'>
        {highlighted_text}
    </div>
    """, unsafe_allow_html=True)

    # Predicted Emotion
    st.markdown(f"<h2 style='color:#FF5733; text-align:center'>Predicted Emotion: {predicted_emotion.upper()}</h2>", unsafe_allow_html=True)

    # Retrieve similar texts
    similar_texts = retrieve_similar_texts(user_input, predicted_emotion)

    # Empathetic reply
    empathetic_reply = generate_empathetic_reply(similar_texts)

    # Bullets
    bullet_html = "<ul style='list-style-type:disc; text-align:left; display:inline-block;'>"
    bullet_html += f"<li>💡 Predicted Emotion: <b>{predicted_emotion.upper()}</b></li>"
    bullet_html += "<li>📊 Model confidence visualization below</li>"
    if similar_texts:
        bullet_html += "<li>🔹 Similar texts from dataset:</li>"
        for txt in similar_texts:
            bullet_html += f"<li style='margin-left:20px'>{txt}</li>"
    bullet_html += f"<li>💡 Empathetic reply based on dataset: {empathetic_reply}</li>"
    bullet_html += "</ul>"
    st.markdown(f"<div style='text-align:center'>{bullet_html}</div>", unsafe_allow_html=True)

    # Model confidence chart
    st.markdown("<h3 style='text-align:center'>Model Confidence Scores</h3>", unsafe_allow_html=True)
    score_df = pd.DataFrame(list(scores.items()), columns=["Label","Score"]).sort_values(by="Score", ascending=False)
    st.bar_chart(score_df.set_index("Label"))

    # 🔊 Speak reply
    tts_file = speak_text(empathetic_reply)
    st.audio(tts_file, format="audio/mp3")

Overwriting app.py


## 🌐 Exposing the Streamlit App with Ngrok

This cell allows us to run the **EmpathyBot Streamlit app** in Google Colab (or any remote environment) and expose it via a public URL using **ngrok**.

**Steps:**

1. **Set your Ngrok Auth Token**  
   Replace the placeholder with your own [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken):
   ```python
   NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"
2. Configure ngrok
3. Run Streamlit in the background
4. Get the public URL

In [16]:
from pyngrok import ngrok, conf

# Replace with your token
NGROK_AUTH_TOKEN = "32W7hIW1xGI3CKHENP4w0v5JuQW_zVF5twNG1TZv2kRHsyh7"

!ngrok config add-authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
from pyngrok import ngrok
!streamlit run app.py &>/dev/null &
url = ngrok.connect(8501)
print('Chatbot running at:', url)

Chatbot running at: NgrokTunnel: "https://34c92949d6fb.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# ugh, missed the bus again and now i’m gonna be late 😩
# omg i got tickets to see my favorite band live next month!! 🤩
# i feel like nobody really listens to me at work…
# just finished a 10k run and feeling unstoppable! 💪🏃‍♂️
# i don’t even know why i bothered… everything went wrong
# can’t believe i ate the whole cake by myself… feeling so guilty 😞
# finally finished my project and it turned out amazing!! 😄
# i just can’t handle all this stress at once
# met my childhood hero today!! still can’t believe it 😍
# had the best day at the beach with friends 🏖️☀️